#Installations

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
%cd /content/drive/MyDrive/MIDI-VAE-NEW

/content/drive/MyDrive/MIDI-VAE-NEW


In [14]:

# Set the path to your MIDI files
midi_folder_path = "/content/drive/MyDrive/MIDI-VAE-NEW/data"

In [15]:
import os
print(f"Folder exists: {os.path.exists(midi_folder_path)}")
print(f"Files in folder: {os.listdir(midi_folder_path)}")

Folder exists: True
Files in folder: ['the_color_of_my_love_jlh.mid', 'too_young-kar_gl.mid', 'the_world_we_knew-kar_rt.mid', 'tin_roof_blues_bcw.mid', 'the_prussian_one_Wade-Culbreath_mlct.mid', 'the_girl_from_ipanema-getz-gilberto1964_dc.mid', 'two_for_the_road-5-slobossa_dm.mid', 'tutu-Miles-Davis_dz.mid', 'tuxedo_junction_jc.mid', 'time_after_time_dm.mid', 'the_gypsy_in_my_soul-kar_jpp.mid', 'victoria_station_am.mid', 'the_goodbye_look-DFagen_h3.mid', 'the_song_is_you_mw.mid', 'the_goodbye_look-Don-Fagen_hir.mid', 'this_guys_in_love_with_you_dc.mid', 'this_here_ccm.mid', 'the_maharajah_of_magador-kar_bpj.mid', 'tranquility_pete-weis.mid', 'the_look_of_love-Diana-Krall-kar_jjd.mid', 'therell_be_life_love_and_laughter-1945-Kurt-Weill-kar_jpp.mid', 'valentine_jim_brickman_cs.mid', 'the_folks_who_live_on_the_hill_dmk.mid', 'those_old_cotton_fields_back-2_home-Pete-Fountain_rw.mid', 'till_there_was_you_ap.mid', 'this_masquerade-GB-LR_oz.mid', 'the_cat_srig.mid', 'tropic_twilight_lg.mid'

In [16]:
#!pip install pretty_midi numpy tensorflow matplotlib

In [17]:
# First, uninstall current versions
#!pip uninstall -y pretty_midi numpy

# Install specific compatible versions
!pip install numpy==1.23.5
!pip install pretty_midi==0.2.9

In [18]:
!pip install tqdm

#MIDI-VAE Class with update Project environemnt (Python 3.11)

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

class MIDI_VAE(keras.Model):
    def __init__(self, sequence_length, input_dim, latent_dim, intermediate_dim):
        """
        Modern implementation of MIDI-VAE using TensorFlow 2.x

        Args:
            sequence_length: Length of the MIDI sequence
            input_dim: Dimension of input features
            latent_dim: Dimension of the latent space
            intermediate_dim: Dimension of the intermediate layers
        """
        super(MIDI_VAE, self).__init__()

        # Encoder
        self.encoder_lstm = layers.LSTM(intermediate_dim,
                                      return_sequences=True)
        self.encoder_lstm2 = layers.LSTM(intermediate_dim,
                                       return_state=True)

        # VAE layers
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)

        # Decoder
        self.decoder_initial = layers.Dense(intermediate_dim,
                                          activation='relu')
        self.decoder_lstm = layers.LSTM(intermediate_dim,
                                      return_sequences=True,
                                      stateful=False)
        self.decoder_dense = layers.Dense(input_dim,
                                        activation='sigmoid')

        # Store parameters
        self.latent_dim = latent_dim
        self.sequence_length = sequence_length

    def encode(self, x):
        # Initial LSTM encoding
        x = self.encoder_lstm(x)

        # Get final states
        _, state_h, state_c = self.encoder_lstm2(x)

        # Generate latent parameters
        z_mean = self.dense_mean(state_h)
        z_log_var = self.dense_log_var(state_h)

        return z_mean, z_log_var

    def reparameterize(self, z_mean, z_log_var):
        batch = tf.shape(z_mean)[0]
        epsilon = tf.random.normal(shape=(batch, self.latent_dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    def decode(self, z):
        # Initial state from latent vector
        initial_state = self.decoder_initial(z)

        # Repeat vector to create sequence
        x = tf.repeat(tf.expand_dims(initial_state, 1),
                     self.sequence_length, axis=1)

        # Decode sequence
        x = self.decoder_lstm(x)
        reconstruction = self.decoder_dense(x)

        return reconstruction

    def call(self, x):
        # Full forward pass
        z_mean, z_log_var = self.encode(x)
        z = self.reparameterize(z_mean, z_log_var)
        reconstruction = self.decode(z)

        # Add KL divergence loss
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)

        return reconstruction

# Training utilities
@tf.function
def train_step(model, x, optimizer):
    """Single training step"""
    with tf.GradientTape() as tape:
        reconstruction = model(x)
        # Reconstruction loss (binary crossentropy)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.binary_crossentropy(x, reconstruction),
                #axis=[1, 2]
                axis=[1] # problematic line corrected
            )
        )
        # Total loss (including KL divergence added in model.call)
        total_loss = reconstruction_loss + tf.reduce_sum(model.losses)

    # Compute and apply gradients
    grads = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return total_loss, reconstruction_loss

# Example usage
def create_and_compile_model(sequence_length=32,
                           input_dim=128,
                           latent_dim=256,
                           intermediate_dim=512):
    """Create and compile the MIDI-VAE model"""
    model = MIDI_VAE(sequence_length, input_dim, latent_dim, intermediate_dim)
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)

    # Build model
    dummy_input = tf.zeros((1, sequence_length, input_dim))
    _ = model(dummy_input)

    return model, optimizer

#MIDI Preprocessing Test Script

In [20]:
import pretty_midi
import tensorflow as tf
import numpy as np
import glob
import os
from collections import defaultdict
import matplotlib.pyplot as plt

class MIDIPreprocessor:
    def __init__(self, sequence_length=32,
                 min_pitch=21, max_pitch=109,
                 time_step=0.125):
        self.sequence_length = sequence_length
        self.min_pitch = min_pitch
        self.max_pitch = max_pitch
        self.time_step = time_step
        self.n_pitches = max_pitch - min_pitch + 1

    def load_midi_file(self, file_path):
        try:
            # Use explicit integer types
            pretty_midi.pretty_midi.MAX_TICK = np.int64(1e7)
            midi_data = pretty_midi.PrettyMIDI(file_path)
            return midi_data
        except Exception as e:
            print(f"Error loading {file_path}: {str(e)}")
            return None

    def midi_to_piano_roll(self, midi_data):
        try:
            # Get piano roll with explicit sampling rate
            fs = int(1/self.time_step)
            piano_roll = midi_data.get_piano_roll(fs=fs)

            # Ensure we have enough pitches
            if piano_roll.shape[0] < self.max_pitch + 1:
                pad_size = self.max_pitch + 1 - piano_roll.shape[0]
                piano_roll = np.pad(piano_roll, ((0, pad_size), (0, 0)))

            # Crop to our pitch range
            piano_roll = piano_roll[self.min_pitch:self.max_pitch + 1]

            # Convert to binary (note on/off)
            piano_roll = (piano_roll > 0).astype(np.float32)

            return piano_roll.T
        except Exception as e:
            print(f"Error in piano roll conversion: {str(e)}")
            return None

    def extract_sequences(self, piano_roll):
        if piano_roll is None:
            return np.array([])

        sequences = []
        for i in range(0, len(piano_roll) - self.sequence_length + 1):
            sequence = piano_roll[i:i + self.sequence_length]
            if np.any(sequence):
                sequences.append(sequence)

        if not sequences:
            return np.array([])

        return np.array(sequences)

    def create_tf_dataset(self, midi_files, batch_size=32, shuffle=True):
        all_sequences = []
        processed_files = 0

        for file_path in midi_files:
            try:
                midi_data = self.load_midi_file(file_path)
                if midi_data is None:
                    continue

                piano_roll = self.midi_to_piano_roll(midi_data)
                if piano_roll is None:
                    continue

                sequences = self.extract_sequences(piano_roll)
                if len(sequences) > 0:
                    all_sequences.append(sequences)
                    processed_files += 1

                if processed_files % 10 == 0:
                    print(f"Processed {processed_files} files...")

            except Exception as e:
                print(f"Error processing {file_path}: {str(e)}")
                continue

        if not all_sequences:
            raise ValueError("No valid sequences were extracted from the MIDI files")

        all_sequences = np.concatenate(all_sequences, axis=0)
        print(f"Created dataset with {len(all_sequences)} sequences")

        dataset = tf.data.Dataset.from_tensor_slices(all_sequences)
        if shuffle:
            dataset = dataset.shuffle(10000)

        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)

        return dataset

def test_midi_preprocessing(midi_folder):
    """
    Test MIDI preprocessing pipeline with improved error handling
    """
    print("=== MIDI Preprocessing Test ===")

    # 1. File Discovery
    print("\n1. Checking MIDI files...")
    midi_files = glob.glob(os.path.join(midi_folder, "*.mid"))
    midi_files.extend(glob.glob(os.path.join(midi_folder, "*.midi")))

    if not midi_files:
        print("❌ No MIDI files found in the specified folder!")
        return None, None

    print(f"✓ Found {len(midi_files)} MIDI files")
    print("\nFirst 5 files:")
    for file in midi_files[:5]:
        print(f"  - {os.path.basename(file)}")

    # 2. Initialize Preprocessor
    print("\n2. Initializing preprocessor...")
    preprocessor = MIDIPreprocessor(
        sequence_length=32,
        min_pitch=21,
        max_pitch=109,
        time_step=0.125
    )
    print("✓ Preprocessor initialized")

    # 3. Try multiple files until one works
    print("\n3. Testing file processing...")
    midi_data = None
    test_file = None

    for file in midi_files[:5]:  # Try first 5 files
        print(f"Attempting to process: {os.path.basename(file)}")
        midi_data = preprocessor.load_midi_file(file)
        if midi_data is not None:
            test_file = file
            break

    if midi_data is None:
        print("❌ Failed to load any test MIDI files!")
        return None, None

    # Continue with the rest of the processing...
    print("\nSuccessfully loaded MIDI file!")
    print(f"Duration: {midi_data.get_end_time():.2f} seconds")

    # 4. Create dataset
    print("\n4. Creating dataset...")
    try:
        dataset = preprocessor.create_tf_dataset(
            midi_files,
            batch_size=32,
            shuffle=True
        )
        print("✓ Dataset created successfully!")
        return dataset, preprocessor
    except Exception as e:
        print(f"❌ Error creating dataset: {str(e)}")
        return None, None

# Run the test
print("Starting MIDI preprocessing test...")

dataset, preprocessor = test_midi_preprocessing(midi_folder_path)

if dataset is not None:
    print("\nDataset creation successful!")
    # Show first batch info
    for batch in dataset.take(1):
        print(f"Batch shape: {batch.shape}")
        print(f"Batch min value: {tf.reduce_min(batch)}")
        print(f"Batch max value: {tf.reduce_max(batch)}")
else:
    print("\nFailed to create dataset. Please check the errors above.")

Starting MIDI preprocessing test...
=== MIDI Preprocessing Test ===

1. Checking MIDI files...
✓ Found 50 MIDI files

First 5 files:
  - the_color_of_my_love_jlh.mid
  - too_young-kar_gl.mid
  - the_world_we_knew-kar_rt.mid
  - tin_roof_blues_bcw.mid
  - the_prussian_one_Wade-Culbreath_mlct.mid

2. Initializing preprocessor...
✓ Preprocessor initialized

3. Testing file processing...
Attempting to process: the_color_of_my_love_jlh.mid

Successfully loaded MIDI file!
Duration: 191.36 seconds

4. Creating dataset...
Processed 10 files...
Processed 20 files...
Processed 30 files...
Processed 40 files...
Processed 50 files...
Created dataset with 85431 sequences
✓ Dataset created successfully!

Dataset creation successful!
Batch shape: (32, 32, 89)
Batch min value: 0.0
Batch max value: 1.0


#VAE Training and Generation Code



In [21]:
import os
import glob

def delete_checkpoint(checkpoint_path):
    """
    Delete all files associated with a checkpoint.

    Args:
        checkpoint_path: Path to the checkpoint (without file extensions).
    """
    # Find all files associated with the checkpoint
    checkpoint_files = glob.glob(f"{checkpoint_path}*")

    # Delete each file
    for file in checkpoint_files:
        os.remove(file)
        print(f"Deleted {file}")

In [22]:
import tensorflow as tf
import numpy as np
import os
import pretty_midi
from datetime import datetime

class VAETrainer:
    def __init__(self, model, checkpoint_dir, optimizer=None):
        self.model = model
        self.checkpoint_dir = checkpoint_dir
        self.optimizer = optimizer or tf.keras.optimizers.Adam(1e-4)

        # Create checkpoint manager
        self.checkpoint = tf.train.Checkpoint(
            model=self.model,
            optimizer=self.optimizer
        )
        self.manager = tf.train.CheckpointManager(
            self.checkpoint,
            checkpoint_dir,
            max_to_keep=3  # Keep 3 latest checkpoints
        )

        # Restore latest checkpoint if it exists
        if self.manager.latest_checkpoint:
            self.checkpoint.restore(self.manager.latest_checkpoint)
            print(f"Restored from checkpoint: {self.manager.latest_checkpoint}")
        else:
            print("Initializing from scratch")

        # Training metrics
        self.train_loss = tf.keras.metrics.Mean(name='train_loss')
        self.reconstruction_loss = tf.keras.metrics.Mean(name='reconstruction_loss')

    @tf.function
    def train_step(self, batch):
        """Single training step"""
        with tf.GradientTape() as tape:
            # Forward pass
            reconstruction = self.model(batch)

            # Reconstruction loss
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(batch, reconstruction),
                    axis=[1]  # Changed from axis=[1, 2] to axis=[1]
                )
            )

            # Total loss (including KL divergence added in model.call)
            total_loss = reconstruction_loss + tf.reduce_sum(self.model.losses)

        # Compute and apply gradients
        grads = tape.gradient(total_loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        # Update metrics
        self.train_loss.update_state(total_loss)
        self.reconstruction_loss.update_state(reconstruction_loss)

        return total_loss, reconstruction_loss



    def train(self, dataset, epochs, steps_per_epoch=None, save_freq='epoch', best_checkpoint_metric='loss'):

        """
        Train the model with optimized checkpointing.

        Args:
            dataset: Training dataset.
            epochs: Number of epochs to train.
            steps_per_epoch: Number of steps per epoch (optional).
            save_freq: Frequency of saving checkpoints. Can be 'epoch' or an integer (steps).
            best_checkpoint_metric: Metric to track for saving the best checkpoint (e.g., 'loss' or 'reconstruction_loss').
        """
        # Initialize variables for tracking the best checkpoint
        best_metric_value = float('inf')  # Assuming we want to minimize the metric
        best_checkpoint_path = None

        for epoch in range(epochs):
            print(f"\nEpoch {epoch+1}/{epochs}")

            # Reset metrics
            self.train_loss.reset_state()
            self.reconstruction_loss.reset_state()

            # Training loop
            step = 0
            for batch in dataset:
                loss, rec_loss = self.train_step(batch)

                if step % 100 == 0:
                    print(f"\rStep {step}: Loss = {loss:.4f}, Reconstruction Loss = {rec_loss:.4f}", end='')

                # Save intermediate checkpoint (if save_freq is in steps)
                if isinstance(save_freq, int) and step % save_freq == 0:
                    save_path = self.manager.save()
                    print(f"\nSaved intermediate checkpoint: {save_path}")

                step += 1
                if steps_per_epoch and step >= steps_per_epoch:
                    break

            # End of epoch
            epoch_loss = self.train_loss.result()
            epoch_rec_loss = self.reconstruction_loss.result()
            print(f"\nEpoch {epoch+1} - Average Loss: {epoch_loss:.4f}, "
                  f"Average Reconstruction Loss: {epoch_rec_loss:.4f}")

            # Save checkpoint at end of epoch
            if save_freq == 'epoch':
                save_path = self.manager.save()
                print(f"Saved checkpoint: {save_path}")

            # Track the best checkpoint
            if best_checkpoint_metric == 'loss':
                metric_value = epoch_loss
            elif best_checkpoint_metric == 'reconstruction_loss':
                metric_value = epoch_rec_loss
            else:
                raise ValueError(f"Unsupported metric: {best_checkpoint_metric}")

            if metric_value < best_metric_value:
                best_metric_value = metric_value
                if best_checkpoint_path:
                    try:
                        print(f"Deleting previous best checkpoint: {best_checkpoint_path}")
                        #os.remove(best_checkpoint_path)  # Delete the previous best checkpoint
                        delete_checkpoint(best_checkpoint_path)
                    except FileNotFoundError:
                        print(f"Warning: Previous best checkpoint not found: {best_checkpoint_path}")
                best_checkpoint_path = save_path
                print(f"New best checkpoint saved: {best_checkpoint_path}")

        # Final message
        print("\nTraining complete!")
        if best_checkpoint_path:
            print(f"Best checkpoint (lowest {best_checkpoint_metric}): {best_checkpoint_path}")


class MusicGenerator:
    def __init__(self, model, preprocessor):
        self.model = model
        self.preprocessor = preprocessor

    def sample_latent_space(self, n_samples=1):
        """Sample from the latent space"""
        return tf.random.normal((n_samples, self.model.latent_dim))

    def interpolate_latent_points(self, z1, z2, steps=10):
        """Interpolate between two points in latent space"""
        alphas = np.linspace(0, 1, steps)
        z_interp = np.array([(1-a)*z1 + a*z2 for a in alphas])
        return z_interp

    def generate_sequences(self, z):
        """Generate sequences from latent vectors"""
        return self.model.decode(z)

    def sequences_to_midi(self, sequences, tempo=120, output_path=None):
        """Convert generated sequences to MIDI file"""
        pm = pretty_midi.PrettyMIDI(initial_tempo=tempo)
        piano = pretty_midi.Instrument(program=0)  # Piano

        current_time = 0.0
        for sequence in sequences:
            for time_step, note_vector in enumerate(sequence):
                for note_idx, is_active in enumerate(note_vector):
                    if is_active > 0.5:  # Note is active
                        # Convert to MIDI note number
                        note_number = note_idx + self.preprocessor.min_pitch

                        # Create note
                        note = pretty_midi.Note(
                            velocity=100,
                            pitch=note_number,
                            start=current_time,
                            end=current_time + self.preprocessor.time_step
                        )
                        piano.notes.append(note)
                current_time += self.preprocessor.time_step

        pm.instruments.append(piano)

        if output_path:
            pm.write(output_path)

        return pm

# Example usage
def setup_training_and_generation(model, dataset, checkpoint_dir, epochs=5):
    """Setup and run training, then demonstrate generation"""
    # Create trainer
    trainer = VAETrainer(model, checkpoint_dir)

    # Train model
    trainer.train(dataset, epochs=epochs, save_freq=1000)

    return trainer

def generate_sample_music(model, preprocessor, output_dir):
    """Generate sample music using the trained model"""
    generator = MusicGenerator(model, preprocessor)

    # Generate from random latent vector
    z = generator.sample_latent_space(1)
    sequences = generator.generate_sequences(z)

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Generate MIDI file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = os.path.join(output_dir, f"generated_music_{timestamp}.mid")
    generator.sequences_to_midi(sequences, output_path=output_path)

    print(f"Generated MIDI file saved to: {output_path}")

    return output_path

#Training with Checkpointing:

In [23]:
# Setup paths
checkpoint_dir = "/content/drive/MyDrive/MIDI-VAE-NEW/checkpoints"
output_dir = "/content/drive/MyDrive/MIDI-VAE-NEW/generated"

# Create and compile model (using our previous VAE model)
model, optimizer = create_and_compile_model(
    sequence_length=preprocessor.sequence_length,
    input_dim=preprocessor.n_pitches,
    latent_dim=256,
    intermediate_dim=512
)

# Setup training
trainer = VAETrainer(model, checkpoint_dir, optimizer)

# Train model
trainer.train(dataset, epochs=45, save_freq='epoch', best_checkpoint_metric='loss')



Restored from checkpoint: /content/drive/MyDrive/MIDI-VAE-NEW/checkpoints/ckpt-138

Epoch 1/45
Step 2600: Loss = 1.1985, Reconstruction Loss = 0.9448
Epoch 1 - Average Loss: 1.1355, Average Reconstruction Loss: 0.8923
Saved checkpoint: /content/drive/MyDrive/MIDI-VAE-NEW/checkpoints/ckpt-139
New best checkpoint saved: /content/drive/MyDrive/MIDI-VAE-NEW/checkpoints/ckpt-139

Epoch 2/45
Step 2600: Loss = 1.3134, Reconstruction Loss = 1.0540
Epoch 2 - Average Loss: 1.1337, Average Reconstruction Loss: 0.8904
Saved checkpoint: /content/drive/MyDrive/MIDI-VAE-NEW/checkpoints/ckpt-140
Deleting previous best checkpoint: /content/drive/MyDrive/MIDI-VAE-NEW/checkpoints/ckpt-139
Deleted /content/drive/MyDrive/MIDI-VAE-NEW/checkpoints/ckpt-139.data-00000-of-00001
Deleted /content/drive/MyDrive/MIDI-VAE-NEW/checkpoints/ckpt-139.index
New best checkpoint saved: /content/drive/MyDrive/MIDI-VAE-NEW/checkpoints/ckpt-140

Epoch 3/45
Step 2600: Loss = 1.1210, Reconstruction Loss = 0.8691
Epoch 3 - Aver

#Generating Music:

In [24]:
# Create generator
generator = MusicGenerator(model, preprocessor)

# Generate from random latent vector
z = generator.sample_latent_space(1)
sequences = generator.generate_sequences(z)

# Save as MIDI
output_path = os.path.join(output_dir, "generated_music.mid")
generator.sequences_to_midi(sequences, output_path=output_path)

#Interpolation between styles:

In [25]:
# Generate two random points in latent space
z1 = generator.sample_latent_space(1)[0]
z2 = generator.sample_latent_space(1)[0]

# Create interpolation
z_interp = generator.interpolate_latent_points(z1, z2, steps=10)
sequences = generator.generate_sequences(z_interp)

# Save as MIDI
output_path = os.path.join(output_dir, "interpolated_music.mid")
generator.sequences_to_midi(sequences, output_path=output_path)

# Convert midi to wav

In [26]:
from IPython.display import Audio

# Convert MIDI to WAV using fluidsynth
!apt-get install fluidsynth
# !wget https://download.sf2tool.com/GeneralUser_GS_1.471.zip
# !unzip GeneralUser_GS_1.471.zip
!wget https://www.philscomputerlab.com/uploads/3/7/2/3/37231621/weedsgm3.sf2


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5
  libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common libwacom9 libxcb-icccm4 libxcb-image0
  libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5
  libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common libwacom9 libxcb-icc

In [27]:
midi_file_path = "/content/drive/MyDrive/MIDI-VAE-NEW/generated/generated_music.mid"
# Convert MIDI to WAV
wav_file_path = "/content/drive/MyDrive/MIDI-VAE-NEW/generated/generated_music.wav"
!fluidsynth -ni weedsgm3.sf2 {midi_file_path} -F {wav_file_path} -r 44100

# The rate argument is added to Audio to specify the sample rate.
Audio(wav_file_path, rate=44100)

FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

fluidsynth: warning: Instrument 'Honkytonk Piano': Some invalid generators were discarded
Rendering audio to file '/content/drive/MyDrive/MIDI-VAE-NEW/generated/generated_music.wav'..


In [28]:
midi_file_path = "/content/drive/MyDrive/MIDI-VAE-NEW/generated/interpolated_music.mid"
# Convert MIDI to WAV
wav_file_path = "/content/drive/MyDrive/MIDI-VAE-NEW/generated/interpolated_music.wav"
!fluidsynth -ni weedsgm3.sf2 {midi_file_path} -F {wav_file_path} -r 44100

# The rate argument is added to Audio to specify the sample rate.
Audio(wav_file_path, rate=44100)